In [1]:
!pip install folium
import requests
import pandas as pd
import folium

In [2]:
# Replace YOUR_API_KEY with your actual Yelp API key
api_key = '7JoimcZI1TN7orq7HjWtZaGPSkBcPMAPWKVD6rsP5zTAlUGHzL7Rhh2rm5hlDEZnbHU0S-LBlPOR-zf44Ti7ece21OSdAjVaehvoxWED0MhaxyeIcqUOYoPo310pZHYx'

# Set the location and search term for the search
location = 'Birmingham, UK'
term = 'jewellery'

# Set the API endpoint and parameters
url = 'https://api.yelp.com/v3/businesses/search'
headers = {
    'Authorization': f'Bearer {api_key}'
}
params = {
    'location': location,
    'term': term,
}

# Send the API request and get the response
response = requests.get(url, headers=headers, params=params)
data = response.json()

stores = pd.DataFrame(columns=['Name', 'Address', 'Latitude', 'Longitude'])
for business in data.get('businesses', []):
    name = business.get('name', '')
    address = ', '.join(business.get('location', {}).get('display_address', []))
    latitude = business.get('coordinates', {}).get('latitude', None)
    longitude = business.get('coordinates', {}).get('longitude', None)
    stores = stores.append({'Name': name, 'Address': address, 'Latitude': latitude, 'Longitude': longitude}, ignore_index=True)

stores

,Name,Address,Latitude,Longitude
0,Nightingale Jewellery,"7 Oak Tree Lane, Selly Oak, Birmingham B29 6JE...",52.439079,-1.939737
1,Uncle's The Original Pawnbrokers,"749 Bristol Road South, Northfield, Birmingham...",52.415967,-1.971049
2,Prem Jewellers,"123 Soho Road, Handsworth, Birmingham B21 9ST,...",52.502258,-1.929724
3,Aman's,"222 Soho Road, Handsworth, Birmingham B21 9LR,...",52.503696,-1.935635
4,Sell Gold Wolverhapton @ Goldcash,"6 hampton walk queens square, Wolverhampton, W...",52.585451,-2.128482
5,Paul Jewellers,"99 High Street, West Bromwich B70 6NZ, United ...",52.515526,-1.988323
6,Uropa Jewellers,"132 Soho Road, Handsworth, Birmingham B21 9LN,...",52.502941,-1.931733
7,Laffaire Exclusive,"244 Soho Road, Handsworth, Birmingham B21 9LR,...",52.503696,-1.935635
8,Gem Box,"1541 Stratford Road, Hall Green, Birmingham B2...",52.421520,-1.836213
9,Cheshires Jewellers,"4a Poplar Arcade, Solihull B91 3GH, United Kin...",52.413464,-1.779558


In [3]:
# Plot the stores on a map using Folium
map_center = [52.489471, -1.898575]  # Birmingham, UK
store_map = folium.Map(location=map_center, zoom_start=10)

# Iterate through each pair of stores and add a link between them
for i, store1 in stores.iterrows():
    for j, store2 in stores.iterrows():
        if i != j:
            # Create a PolyLine between the two stores
            line = folium.PolyLine(
                locations=[store1[['Latitude', 'Longitude']].values.tolist(),
                           store2[['Latitude', 'Longitude']].values.tolist()],
                color='blue',
                weight=2
            )
            line.add_to(store_map)
    
    # Add a marker(node) for the store
    store_name = store1['Name']
    store_address = store1['Address']
    store_location = [store1['Latitude'], store1['Longitude']]
    popup_text = f"{store_name}<br>{store_address}"
    folium.Marker(location=store_location, popup=popup_text).add_to(store_map)
    
store_map